## Wiener Diarum Sterbelisten
-----------------------------------------------------------------------------------------------

### Data

The digitalised Wiener Zeitung is partially available as TEI XML as well as an extracted HTML on Github. Starting from 1703, there is representative data.This data set contains only a few newspapers per year starting from 1706.

### Task

Filter out Toponyms from the Sterbelisten. For this sequence labeling task we need to first tokenize the words and use the rules (patterns and keywords) made by the team (Nina) to fit to our classes (Toponym or not).


##### What we have 
There is a list with sucessfully extracted toponyms, which are already normalised orthography.
There is a list with rules which are combines Regex and other expressions. 
There is an html file that has \<mark> on most toponyms but sometimes on persons and other names too.
There are defined classes (Toponym/no Toponym).
There are .csv files with Toponyms extracted from Wien Geschichte Wiki.

##### What we want
A trainig and test set containing three matrixes:
T Matrix containing the classes and the rules.(np.matrix, sparse)
Z Matrixes each containing the samples and the rules.(np.matrix, sparse)
X Matrix containing the samples (data frame)


##### What to do

**1. Read in the html file**
Strip of tags and clean the text with re
Expand abbreviations- if possible?
create a list with each sentence as an item

**2. Read in already extracted placenames**
Extract all \<mark> words from HTML**
Using their code
clean the placenames
create a set from exsiting placenames

**3. Building training data**
As there is no clean manual training data, it has to be build:

**X Matrx**
list with all sentenses

maybe spacy?

**T Matrix**

np.sparse matirx 1 colum 0, 1 colum 1, rows are keywords

**Z Matrices**

Each sentense is going to be its own Z Matrix. 
create a list with each word as an item
create a list(/dict if index) with bigrams as items
create a list(/dict if index) with trigrams as items
create a data frame
colums= keywords 
rows = words in sentense



**5. Building test data** 

manually checked! 
There is the 'timemachine_evaluatuin_v1_edited_corrected.jsonl which should contain manually corrected place names but I don't understand where the actual word is tagged or how to filter out that information.

### 1. Read in HTML

In [20]:
'''Import libraries and load html file'''

import os
import re
import codecs

file = codecs.open('annotations_3-21_v4.html', "r", "utf-8")
sterbelisten_html = file.read()

'''Cleaning the text first from both html taggs and xml tags and than in a second step clean the intro and replace with /n'''

sterbelisten_strip_html_1 = re.sub(r"<h2>.*</h2><h3>.*xml \|.*\d+</h3><br/>","",sterbelisten_html)

sterbelisten_strip_html1 = re.sub(r"<hr/>|<p>|</p>|#+|=| =|<html>|</html>|<mark>|</mark>|\r|/|\b \.|\b# \.","",sterbelisten_strip_html_1)

sterbelisten_strip_html2 = re.sub(r"    \b","",sterbelisten_strip_html1)

In [21]:
'''create an empty list and split the text at line break, creating von element for each sentense'''

sterbeliste = []
sterbeliste = sterbelisten_strip_html2.split("\n\n\n")
print(sterbeliste)

'''create a list with words for each sentence'''
sterbeliste_words= [i.split(" ") for i in sterbeliste]

'''removing empty objects in sterbeliste_words'''
#sterbeliste_words.remove('')
#sterbeliste_words = [word for sentense in sterbeliste_words list(filter(None, i))]
#sterbeliste_words= [i for i in sterbeliste_words if i!=' ']
#sterbeliste_words= [word for word in sterbeliste_words if not "re.search(\s)"]
'''
Problems
find a way to filter out the rest of the "" elements in the lists.
'''
#print(sterbeliste_words)

['\nDem Peter Frost  einem Cammer im Greiseckeris  Hauß im Diener  Gäßl  sein Kind Frantz  alt 6 viertl Jahr ', 'Der Maria Nauitschanin  einer Burgerl Wittib im Primis  Hauß auf der Wendelstadt  ihr Kind Carl  alt 5 Jahr ', 'Christina Kochin  ein Wittib im Barbieris  Hauß auf der Laimgrueben  alt 67 Jahr ', 'Gottlieb Rabel  ein gewester Haußmeister beyn 3  Mohren in der Roßau  alt 52 Jahr ', 'Dominica Stephanebrin  ein lediges Mensch beym weissen Ochsen bey St  Ulrich  alt 17 Jahr ', 'Mathias Rausch  ein Fuhrknecht beyn 3  Hertzen am Spittelberg  alt 18 Jahr ', 'Barbara N . ein arme Wittib  ins   Hauß in der Leopold  Stadt  alt 86 Jahr ', 'Dem Johann Molsheim  ein Handelsman beyn drey Haasen bey St  Ulrich  sein Kind Johann  alt 5 Jahr ', 'Dem Simon Deix  eine Mühlner im Perschackischen Hauß in der Unger  Gassen  sein Kind Adam  alt ein halb Jahr ', 'Dem Sebastian Stim  eine Schuhmacher im Frey  Gut auff der Wyden  sein Weib Margareth  alt 54 Jahr ', 'Die Eva Gallin  ein arme Wittib  i

'\nProblems\nfind a way to filter out the rest of the "" elements in the lists.\n'

## 2. Create list with keywords from HTML

In [4]:
'''rules by using Keywords'''
from io import StringIO
import os
from lxml import etree

""" Get all the possible toponyms from annotations*.html file """

file_path = os.path.join(".", "annotations_3-21_v4.html")

# The annotations document is not well-formed as XML, so we need to use the HTMLParser
parser = etree.HTMLParser()

with open(file_path, encoding='utf-8') as f:
    raw_html = f.read()

# Parse the raw_html
tree = etree.parse(StringIO(raw_html), parser)

root = tree.getroot()
marks = root.xpath("//mark")

length = len(marks)

# Construct a set of toponyms (thus removing duplicates)
toponym_strings = {mark.text for mark in marks}

'''Create a list with all words that were marked with mark'''

toponymstring=[]

# clean the strings-> does no work yet
for mark in marks:
        toponym= re.sub(r"#+|=| =|-|/|\.|\d+\s+J","",mark.text) ### why does this not filter out f.ex. "Laimgr   50  J"  
        toponymstring.append(toponym)
print(len(toponymstring))

# create a set so that the iteration later is easier
toponym_set= set(toponymstring)
len(toponym_set)

14870


7226

## 3.Building training data

In [22]:
'''
X Matrix
Dimensions sentesesxsentenses (one column sample df, len(sample))

'''

import pandas as pd

X = pd.DataFrame(sterbeliste)


**T Matrix**

np.sparse matirx 1 colum 0, 1 colum 1, rows are keywords


In [42]:
'''
T Matrix
Dimensions keywords x 2

'''
import numpy as np
a= np.zeros(len(toponym_set))
b= np.full(len(toponym_set),1)

T = np.stack((b,a.T))
T.T.shape


(7226, 2)

**Z Matrices**

Each sentense is going to be its own Z Matrix. 
create a list with each word as an item
create a list(/dict if index) with bigrams as items
create a list(/dict if index) with trigrams as items
create a data frame
colums= keywords 
rows = words in sentense

In [45]:
'''building Z Matrices'''


for sentense in sterbeliste: 
    words = [sentense.split(" ")]
    #bi_gram={w1,w2:k for k,v in words}
    #bi_gram={}
    #for w1,w2 in zip(words[:-1],words[1:]):
        #print(w1,w2)
    #for i, i+1 in range(len(words)): ### fist iterate through the index! 
        #words[i], words[i+1]
        #bi_gram.append(i,i+1)    
    #tri_gram ={}
    Z_S=np.zeros((len(words),len(toponym_set))
        for keyword in toponym_set:
            kw= [keyword.split("")]
            if len(kw) == 1
                for w in words:
                    if w == kw
                         match_w= words[key]
                         Z_S(kw,w)=1
            if len(kw) == 2 
                 for bigram in bigrams:
                     if bigram == kw[0]+' '+kw[1]
                         match_w= bigram[keys]
                         Z_S((kw[0],kw[1]),w[0],w[1])= 1
    
    
    

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_452/903126129.py, line 17)

In [ ]:
### create a tokenized sentenses with spacy
import spacy 
from spacy.lang.de.examples import sentences

nlp = spacy.load("de_core_news_sm")

#for i in sterbeliste:
    #doc = nlp(i)
    #print(doc.text)
    #for token in doc:
        #print(token.text, token.pos_, token.dep_)

## 5. Building test data

In [14]:
'''Loading manually checked test data'''

with jsl.open('timemachine_evaluation_v1_edited_corrected.jsonl','r') as reader: #read in jsonl with jsonlines
    df = pd.json_normalize(reader) 

### I don't understand where the tagged toponym is or whether there is any

NameError: name 'jsl' is not defined